
This challenge serves as final project for the "How to win a data science competition" Coursera course.

In this competition you will work with a challenging time-series dataset consisting of daily sales data, kindly provided by one of the largest Russian software firms - 1C Company.

We are asking you to predict total sales for every product and store in the next month. By solving this competition you will be able to apply and enhance your data science skills.

Load libraries

In [ ]:
#load modules
import os
import pandas as pd
import numpy as np
from pandas import read_csv
from pandas import datetime

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))



Load datasets

In [ ]:
#Load datasets
train=pd.read_csv("/kaggle/input/competitive-data-science-predict-future-sales/sales_train.csv")
test=pd.read_csv("/kaggle/input/competitive-data-science-predict-future-sales/test.csv")
sample=pd.read_csv("/kaggle/input/competitive-data-science-predict-future-sales/sample_submission.csv")
items=pd.read_csv("/kaggle/input/competitive-data-science-predict-future-sales/items.csv")
item_cat=pd.read_csv("/kaggle/input/competitive-data-science-predict-future-sales/item_categories.csv")
shops=pd.read_csv("/kaggle/input/competitive-data-science-predict-future-sales/shops.csv")

Convert date to datetime format

In [ ]:
#convert date to datetime format
train['date'] = pd.to_datetime(train['date'],format = '%d.%m.%Y')
train

Create dataset

In [ ]:
#create pivot table
dataset = train.pivot_table(index = ['shop_id','item_id'],values = ['item_cnt_day'],columns = ['date_block_num'],fill_value = 0,aggfunc='sum')
dataset.reset_index(inplace = True)
dataset

Preprocess data

In [ ]:
ID = test.ID
ID

In [ ]:
test = test.drop(['ID'], axis=1)
test

In [ ]:
#merge pivot table with test set
dataset = pd.merge(test,dataset,on = ['item_id','shop_id'],how = 'left')
dataset

In [ ]:
#check for any null values
dataset.isnull().sum().sum()

In [ ]:
#fill all NaN values with 0
dataset.fillna(0,inplace = True)
dataset.isnull().sum().sum()

In [ ]:
#drop shop_id and item_id
dataset.drop(['shop_id','item_id'],inplace = True, axis = 1)
dataset

Create X, y and X_test

In [ ]:
#split the dataset in two
# the last column is our label
y_train = dataset.iloc[:,-1:]
#drop last column of data
X_train = dataset.iloc[:, :-1]
#drop first colum of data
X_test = dataset.iloc[:,1:]
# lets have a look on the shape 
print(X_train.shape,y_train.shape,X_test.shape)

Define model

An AdaBoost regressor is a meta-estimator that begins by fitting a regressor on the original dataset and then fits additional copies of the regressor on the same dataset but where the weights of instances are adjusted according to the error of the current prediction. 

In [ ]:
from sklearn.ensemble import AdaBoostRegressor
from sklearn.ensemble import RandomForestRegressor

model = AdaBoostRegressor(base_estimator = RandomForestRegressor(max_depth=10), random_state=0, n_estimators=3000).fit(X_train, y_train)
y_pred = model.predict(X_train)
y_pred

In [ ]:
print(model.score(X_train, y_train))

Make predictions

In [ ]:
pred = model.predict(X_test)
pred

Submit file

In [ ]:
# creating submission file 
submission_file = pred
# we will keep every value between 0 and 20
submission_file = submission_file.clip(0,20)
# creating dataframe with required columns 
submission_trp = pd.DataFrame({'ID':ID,'item_cnt_month':submission_file.ravel()})
# creating csv file from dataframe
submission_trp.to_csv('submission.csv',index = False)
submission_trp.head(20)